In [2]:
import numpy as np

In [6]:
data_path='/media/maria/DATA1/Documents/NeuroMatchAcademy2020_dat/unzipped_files/Moniz_2017-05-15.tar/'
trials_intervals = np.load(data_path+'/'+'trials.intervals.npy') # in seconds
spike_times = np.load(data_path+'/'+'spikes.times.npy') * 1000
spike_clusters = np.load(data_path+'/'+'spikes.clusters.npy')
clusters_annotation = np.load(data_path+'/'+'clusters._phy_annotation.npy')

In [7]:
def bin_spikes(spike_times,spike_clusters, clusters_annotation, bin_size = 10):

    # Using clusters._phy_annotation.npy obtain valid clusters (i.e. >= 2)
    # valid_clusters_idx = np.array(np.where(clusters_annotation>=2))[0]

    spike_time_cells = np.empty(len(clusters_annotation), dtype=object) # Initalise empty object
    for i in (np.arange(len(np.unique(spike_clusters)))):
      # Create a spike time arrays, where each array in the array is a spike time of a cell
      spike_time_cells[i] = spike_times[(np.where(spike_clusters == i)[0])]

    # Bin spike times into 10ms intervals
    spike_time_binned = np.empty(len(np.unique(spike_clusters)), dtype=object) # Initalise empty object
    sum_spikes = np.empty(len(np.unique(spike_clusters)), dtype=object) # Initalise empty object

    for cell_num in np.arange(len(spike_time_cells)):
        spike_time_hist = np.histogram(spike_time_cells[cell_num],bins = np.arange(0,np.floor(spike_time_cells[cell_num][-1]),bin_size))
        spike_time_binned[cell_num] = spike_time_hist[0]
        sum_spikes[cell_num] = np.sum(spike_time_binned[cell_num])

    cell_spikes_max = np.argmax(sum_spikes) # cell with the maximum number of spikes for plotting purposes
    # Spike_time_binned returns binned spikes sorted into cells
    # Spike_time_cells returns UNbinned spikes sorted into cells
    # cell_spikes_max returns a single cell index that has the max number of spikes (i.e most active cell)
    return spike_time_binned, spike_time_cells, cell_spikes_max

def sort_cells_trials(spike_time_binned,spike_time_cells, trials_intervals, bin_size = 10):
    # Epoch duration is defined as the period after the visual stimulus

    # Sort into trials
    spike_time_binned_trial = np.empty(len(spike_time_binned), dtype=object)
    for cell_num in np.arange(len(spike_time_binned)):
        spike_time_binned_trial[cell_num] = np.empty(len(trials_intervals), dtype=object)

        for i,trials_start_end in enumerate(trials_intervals):
            # Sort spikes into their trial numbers.
            spike_time_binned_trial[cell_num][i] = spike_time_binned[cell_num][ int(np.floor(trials_start_end[0]*(1000/bin_size))) : int(np.floor(trials_start_end[1]*(1000/bin_size)))]
    
    # spike_time_binned_trial returns spikes that are sorted into cells and trials
    # spike_time_binned_trial_response returns spikes that are sorted into cells and trials, and spliced accordingly to desired epoch duration post-visual stim onset

    return spike_time_binned_trial

In [9]:
spike_time_binned, spike_time_cells, cell_spikes_max=bin_spikes(spike_times,spike_clusters, clusters_annotation,bin_size = 10)
spike_time_binned_trial=sort_cells_trials(spike_time_binned,spike_time_cells, trials_intervals, bin_size = 10)

In [16]:
spike_time_binned_trial[2]

array([array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
              0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
              0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
              0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
              0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
              0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
              0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
              0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
              0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
              0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
              0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
              0, 0, 0, 0, 0, 0, 0, 0])                                         ,
       array([0, 0, 0, 0, 0, 0, 0, 0, 0

In [25]:
def get_rates_for_trials(spike_time_binned_trial):
    rates=[]
    for trial in range(0,247):
        lst=[]
        for cell in range(spike_time_binned_trial.shape[0]):
            lst.append(np.sum(spike_time_binned_trial[cell][trial])/(10*spike_time_binned_trial[cell][trial].shape[0]/1000))
        rates.append(lst)
    return rates


In [26]:
rates=get_rates_for_trials(spike_time_binned_trial)

In [29]:
rates.shape

(247, 806)

In [31]:
np.where(np.isnan(rates)==True)

(array([], dtype=int64), array([], dtype=int64))

In [32]:
np.save('rates_for_poisson.npy',rates)

In [ ]:
def prep_data_for_pca(spike_time_binned_trial):
    pass

In [ ]:
def conduct_pca_get_features(spike_time_binned_trial):
    pass